In [ ]:
#|default_exp art

In [ ]:
#|hide
import nbdev

from pathlib import Path
from __future__ import annotations
from fastcore.test import *
from nbdev.showdoc import *
from pdb import set_trace

# Todo: move the google code in utils module
try:
    from google.colab import drive
    ON_COLAB = True
    print('Running on colab')
    print('Installing wandb and project code')
    !pip install -U git+https://github.com/vtecftwy/metagenomics.git@refactor_cnn_virus
    !pip install -qqU wandb
    
    # Assumes shared gdrive dir accessible through shortcut `Metagenomics` under the root of gdrive.     
    drive.mount('/content/gdrive')
    p2drive = Path('/content/gdrive/MyDrive/Metagenonics')
    p2data =  p2drive / 'CNN_Virus_data'
    assert p2drive.is_dir()
    assert p2data.is_dir()

except ModuleNotFoundError:
    ON_COLAB = False
    print('Running locally')
    try:
        import metagentools
    except ModuleNotFoundError:
        raise ModuleNotFoundError('Cannot find package metagentools. Make sure you pip -e install it in your environment')
    p2data = Path('../data/CNN_Virus_data').resolve()
    assert p2data.is_dir()

%load_ext autoreload
%autoreload 2

Running locally
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#|export
# Imports all dependencies
import numpy as np
import os

from pathlib import Path

# `art`

> Operate `art-illumina` next gen read simmulation package from python

`ART` is an open source package simmulation next generation read of genomes, available on the website of the National Institute of Environmental Health Sciences [here](https://www.niehs.nih.gov/research/resources/software/biostatistics/art/index.cfm). It is a command line interface package. This module makes the package accessible from a jupiter notebook

In [ ]:
! ls -al

/usr/bin/zsh: /home/vtec/miniconda3/envs/metagentools/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
total 92
drwxr-xr-x  4 vtec vtec  4096 Nov 23 17:19 .
drwxr-xr-x 16 vtec vtec  4096 Nov 23 17:04 ..
drwxr-xr-x  2 vtec vtec  4096 Nov 23 17:13 .ipynb_checkpoints
-rw-r--r--  1 vtec vtec     0 Oct 29 23:15 .last_checked
-rw-r--r--  1 vtec vtec  7894 Nov 22 10:14 00_core.ipynb
-rw-r--r--  1 vtec vtec 28965 Nov 23 17:13 01_wandb.ipynb
-rw-r--r--  1 vtec vtec 12426 Nov 23 17:19 02_art.ipynb
-rw-r--r--  1 vtec vtec   290 Oct 29 16:13 _quarto.yml
drwxr-xr-x  2 vtec vtec  4096 Nov 21 16:38 data_dev
-rw-r--r--  1 vtec vtec   593 Oct 29 20:04 index.ipynb
-rw-r--r--  1 vtec vtec   261 Nov 23 17:05 nbdev.yml
-rw-r--r--  1 vtec vtec    99 Nov 23 17:05 sidebar.yml
-rw-rw-r--  1 vtec vtec   600 Sep  6 00:31 styles.css


In [ ]:
! conda info

/usr/bin/zsh: /home/vtec/miniconda3/envs/metagentools/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)

     active environment : metagentools
    active env location : /home/vtec/miniconda3/envs/metagentools
            shell level : 2
       user config file : /home/vtec/.condarc
 populated config files : /home/vtec/.condarc
          conda version : 22.9.0
    conda-build version : not installed
         python version : 3.9.12.final.0
       virtual packages : __cuda=12.0=0
                          __linux=5.10.102.1=0
                          __glibc=2.31=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /home/vtec/miniconda3  (writable)
      conda av data dir : /home/vtec/miniconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/fastchan/linux-64
                          https://conda.anaconda.org/fastchan/noarch
                          h

In [ ]:
#| hide
nbdev.nbdev_export()